# 비짓제주 크롤링 코드

비짓제주 숙박/관광지/F&B/쇼핑 데이터 크롤링 코드


In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import warnings
import pandas as pd

In [3]:
warnings.filterwarnings(action='ignore')

In [4]:
base_url = 'https://www.visitjeju.net/kr/detail/list?menuId=DOM_000001707000000000&cate1cd=cate0000000004#p1&pageSize=12&sortListType=reviewcnt&viewType=thumb&isShowBtag&tag' #보여지는 형태를 꼭 네모 네개 짜리로 바꿀 것

In [5]:
driver = webdriver.Chrome(executable_path = 'D:\Python_417\chromedriver')
driver.get(url=base_url)
# implicit Waits
driver.implicitly_wait(time_to_wait = 5)

In [6]:
cleaning = re.compile('<p.*="">')

In [7]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
places = soup.select('dt>a>p.s_tit') # 제목
places = [cleaning.sub('', str(name))[:-4] for name in places]
# plc_tag = soup.find_all('p', class_= 'item_tag prev')
# plc_tag = [(cleaning.sub('', str(tags))[:-4]).strip() for tags in plc_tag]
# plc_tag는 안에 들어가면 더 있기 때문에 들어가서 구하는 걸로 하자

In [8]:
tmp = driver.find_element_by_css_selector('#content > div > div.cont_wrap > div.recommend_area > ul > li:nth-child(1) > dl > dt > a')
# 여기의 li:nth-child  (1) 안의 숫자를 바꿔가면서 클릭하면 됨
tmp.click()

## (1) 비짓제주 숙박 데이터 크롤링

In [9]:
def click_get_info():
    twice = False
    try:
        name = driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.sub_info_title > h3').text
        stars = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.score_area_l > p').text[
              9:-1]
        tags = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.tag_area').text.replace(
            '\n', ' ')
        address = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(2) > p.info_sub_cont').text
        telephone = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(3) > p.info_sub_cont').text
        like = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(1) > button > p.appraisal_cnt').text
        heart = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(2) > button > p.appraisal_cnt').text
        numreview = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(3) > p.appraisal_cnt').text
        view = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(6) > p.appraisal_cnt').text
        ### 상세정보
        detail_text = driver.find_element_by_css_selector(
            '#tab0 > div').text.replace('\n', ' ')
        ### 추천콘텐츠
        #### 관광지
        rec_tour = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_contents').text.split('\n')[0:-1:2]
        #### 음식점
        try:
            driver.find_element_by_css_selector(
                '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_category > ul > li.add2020_detail_restaurant > a').click()
            rec_restaurant = driver.find_element_by_css_selector(
                '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_contents').text.split('\n')[0:-1:2]
        except:
            driver.find_element_by_css_selector('#tab0 > h2').click()
            driver.find_element_by_css_selector(
                '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_category > ul > li.add2020_detail_restaurant > a').click()
            rec_restaurant = driver.find_element_by_css_selector(
                '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_contents').text.split('\n')[0:-1:2]
            twice=True
        #### 숙박
        driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_category > ul > li.add2020_detail_lodgment > a').click()
        rec_lodgment = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_contents').text.split('\n')[0:-1:2]
        #### 빅데이터
        driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_category > ul > li.add2020_detail_bigdata > a').click()
        rec_bigdata = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_contents').text.split('\n')[0:-1:2]
        ### 이용안내
        detail_side_box = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(2) > div > dl').text.split('\n')
    except:
        name = driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.sub_info_title > h3').text
        stars = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style2 > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.score_area_l > p').text[9:-1]
        tags = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style2 > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.tag_area').text.replace('\n', ' ')
        tags+=' '+driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.tag_area > p:nth-child(2)').text.replace('\n','')
        address = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style2 > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(2) > p.info_sub_cont').text
        telephone = driver.find_element_by_css_selector(
             '#content > div.cont.detail_page.detail_style2 > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(3) > p.info_sub_cont').text
        ## 좋아요
        like = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > ul > li:nth-child(1) > button > p.appraisal_cnt').text
        ### 찜하기
        heart = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > ul > li:nth-child(2) > button > p.appraisal_cnt').text
        ### 리뷰
        numreview = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > ul > li:nth-child(3) > p.appraisal_cnt').text
        ### 조회
        view = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > ul > li:nth-child(6) > p.appraisal_cnt').text
        ### 상세정보
        detail_text = driver.find_element_by_css_selector('#tab0 > div').text.replace('\n', ' ')
        ### 추천콘텐츠
        #### 관광지
        rec_tour = driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_contents').text.split('\n')[0:-1:2]
        #### 음식점
        try:
            driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_category > ul > li.add2020_detail_restaurant > a').click()
            rec_restaurant = driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_contents').text.split('\n')[0:-1:2]
        except:
            driver.find_element_by_css_selector('#tab0 > h2').click()
            driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_category > ul > li.add2020_detail_restaurant > a').click()
            rec_restaurant = driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_contents').text.split('\n')[0:-1:2]
            twice=True
        #### 숙박
        driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_category > ul > li.add2020_detail_lodgment > a').click()
        rec_lodgment = driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_contents').text.split('\n')[0:-1:2]
        #### 빅데이터
        driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_category > ul > li.add2020_detail_bigdata > a').click()
        rec_bigdata = driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_contents').text.split('\n')[0:-1:2]
        ### 이용안내
        detail_side_box = driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(2) > div > dl').text.split('\n')

    detail_side_box = { detail_side_box[i] : detail_side_box[i+1] for i in range(0,len(detail_side_box), 2) }

    return {'name' : name, 'stars' : stars, 'tags': tags, 'address' : address, 'telephone' : telephone, 'like' : like, 'heart' : heart, 'numreview' : numreview, 'view' : view, 'detail_text' : detail_text, 'rec_tour' : rec_tour, 'rec_restaurant' : rec_restaurant, 'rec_lodgment' :  rec_lodgment, 'rec_bigdata' : rec_bigdata, 'detail_side_box' : detail_side_box}, twice

In [24]:
driver.get(url=base_url)
driver.maximize_window()
driver.implicitly_wait(time_to_wait = 5)
cleaning = re.compile('<p.*="">')
final_df = pd.DataFrame(columns = ['name', 'stars', 'tags', 'address', 'telephone', 'like', 'heart', 'numreview', 'view', 'detail_text', 'rec_tour',
                                   'rec_restaurant', 'rec_lodgment', 'rec_bigdata', 'detail_side_box'])
page_number = int(driver.find_element_by_css_selector('#content > div > div.cont_wrap > div.recommend_area > div.theme_tit_area.clear > div > div > div.counter > p > strong').text)//12

In [22]:
def crawling_tour(driver, final_df):
    places = [element.text.split('\n')[1] for element in driver.find_elements_by_css_selector('#content > div > div.cont_wrap > div.recommend_area > ul > li')]
    for i in range(1,len(places)+1):
        place_click = driver.find_element_by_css_selector(f'#content > div > div.cont_wrap > div.recommend_area > ul > li:nth-child({i}) > dl > dt > a')
        place_click.click()
        driver.implicitly_wait(time_to_wait = 20)

        tmp_info_dict, twice = click_get_info()
        tmp_df = pd.DataFrame(columns = tmp_info_dict.keys(), data = [tmp_info_dict.values()])
        final_df = final_df.append(tmp_df)

        driver.execute_script("window.history.go(-1)")
        if twice==True:
            driver.execute_script("window.history.go(-1)")
    # driver.execute_script("window.history.go(-1)")
    return final_df

In [84]:
for i in range(1,page_number):
    final_df = crawling_tour(driver, final_df)
    driver.find_element_by_css_selector('#paging > a.spr_com.page-next').click()
    driver.implicitly_wait(time_to_wait=10)
    print(f'{i}번째 페이지 끝')
    final_df

52번째 페이지 끝
53번째 페이지 끝
54번째 페이지 끝
55번째 페이지 끝
56번째 페이지 끝
57번째 페이지 끝
58번째 페이지 끝
59번째 페이지 끝
60번째 페이지 끝
61번째 페이지 끝
62번째 페이지 끝
63번째 페이지 끝
64번째 페이지 끝
65번째 페이지 끝
66번째 페이지 끝
67번째 페이지 끝
68번째 페이지 끝
69번째 페이지 끝
70번째 페이지 끝
71번째 페이지 끝
72번째 페이지 끝
73번째 페이지 끝
74번째 페이지 끝
75번째 페이지 끝
76번째 페이지 끝
77번째 페이지 끝


In [93]:
final_df = final_df.drop_duplicates('name')

In [87]:
# store_df = pd.read_csv('VISITJEJU_숙소정보크롤링.csv')

In [97]:
store_df = store_df.append(final_df)
store_df = store_df.drop_duplicates('name')
store_df.reset_index(drop=True, inplace=True)

In [98]:
#store_df.to_csv('data/VISITJEJU_숙박정보크롤링_FINAL.csv', encoding='utf-8-sig', index=False)

### 크롤링 안된 경우 체크

In [50]:
#store_df = pd.read_csv('bigcon_data/VISITJEJU_숙박정보크롤링.csv')

In [ ]:
driver.get(url = base_url)

In [56]:
main_url = base_url

In [57]:
for i in range(1,80):
    driver.get(url = main_url)
    places = [element.text.split('\n')[1] for element in driver.find_elements_by_css_selector('#content > div > div.cont_wrap > div.recommend_area > ul > li')]
    print(places)
    main_url = driver.current_url
    for idx, place in enumerate(places):
        if place not in store_df['name'].tolist():
            place_click = driver.find_element_by_css_selector(f'#content > div > div.cont_wrap > div.recommend_area > ul > li:nth-child({idx+1}) > dl > dt > a')
            place_click.click()
            driver.implicitly_wait(time_to_wait = 20)

            tmp_info_dict, twice = click_get_info()
            tmp_df = pd.DataFrame(columns = tmp_info_dict.keys(), data = [tmp_info_dict.values()])
            store_df = store_df.append(tmp_df)

            driver.execute_script("window.history.go(-1)")
            if twice==True:
                driver.execute_script("window.history.go(-1)")

            print(place)
        else:
            pass
    driver.get(url = main_url)
    driver.find_element_by_css_selector('#paging > a.spr_com.page-next').click()
    driver.implicitly_wait(time_to_wait=60)
    main_url = driver.current_url
    print(f'{i}번째 페이지 끝')

['나미송 머무는 곳', '해비치호텔앤드리조트', '신라호텔제주', '롯데호텔 제주', '제주조천스위스마을게스트하우스', '비젠빌리지', '제주도푸른산푸른바다', '휘닉스 제주 섭지코지', '히든클리프 호텔 앤 네이쳐', '대명리조트 제주', '서귀포KAL호텔', '제주도푸른바다']
1번째 페이지 끝
['캠핑트리펜션', '씨에스호텔앤리조트', '객의하우스', '금호리조트제주', '수상한소금밭게스트하우스', '달달하우스', '신라스테이 제주', '틸다하우스', 'inn jeju 인제주 게스트하우스', '르페도라', '제주가좋아서 펜션', '나날 게스트하우스']
2번째 페이지 끝
['라마다프라자 제주호텔', '롯데시티호텔제주', '메종글래드 제주', '제주bp게스트하우스 JEJU BP GUESTHOUSE', '제주부영호텔엔리조트', '빠담빠담게스트하우스', '오션스위츠호텔 제주', '우도사랑', '제주해나루', '켄싱턴리조트 제주 한림점', '활엽수 게스트하우스', '꽃하루방게스트하우스']
3번째 페이지 끝
['롯데리조트아트빌라스', '마레보 리조트', '메종드오조락', '범섬풍경', '서귀포푸른바다', '아름다운리조트', '아이비테라스', '야호비치하우스', '오션스퀘어', '오크하우스', '제주WE호텔', '제주리조트']
4번째 페이지 끝
['제주아이브리조트', '하워드존슨 제주 연동 호텔', '꿈꾸는섬 게스트하우스', '더본 호텔', '더포그레이스호텔앤리조트', '랜딩관 제주신화월드 호텔앤리조트', '베니키아 아이진호텔', '베스트웨스턴제주호텔', '블루하와이리조트', '소풍(월정리)', '예이츠산장', '유월그리고열두마루']
소풍(월정리)
5번째 페이지 끝
['이디하우스n카페', '일레인호텔', '제주인호텔', '제주하이킹 INN(구덕게스트하우스)', '파르나스 호텔 제주', '펄리플러스', '포도호텔', '플레이스 캠프 제주', '하도리일번지', '한화리조트 제주', '게으른소나기', '구름밭하우스']
제주하이킹 INN(구덕게스트하우스)
6번째 페이지 끝

In [ ]:
store_df['name'].value_counts()

In [ ]:
store_df[store_df['name'] == '해뜨는집']

- 해뜨는집이 visitjeju 숙박에 두개 있는데 두개가 하나는 남원, 하나는 도두에 위치하고 있어서 다르게 인식됨 그런데 이름에는 해뜨는집으로 공통되게 나오는 문제가 있음
- 그래서 하나를 해뜨는집 (남원)으로, 다른 걸 해뜨는집 (도두)로 바꿔주려고 함

In [93]:
store_df.at[282, 'name'] = '해뜨는집 (남원)'

In [95]:
store_df.reset_index(drop=True, inplace=True)
store_df[store_df['name'] == '해뜨는집']

,name,stars,tags,address,telephone,like,heart,numreview,view,detail_text,rec_tour,rec_restaurant,rec_lodgment,rec_bigdata,detail_side_box
953,해뜨는집,,#펜션 #숙소 #휴양펜션 #독채 #자연경관 #정원,제주특별자치도 서귀포시 남원읍 하신로 31,(+82) 010-8861-7651,0,1,0,"1,029","1, 2층 전층을 다 이용하는 단독형 펜션이다. 1층에는 주방, 거실, 침실이 있고...","[남원 위미2리 어촌계, 사려니오름, 헌마공신김만일기념관]","[유나카페, 로드3.7 (road3.7), 육값하네식육가든]","[노블렛휴양펜션, 미소가펜션, 부영CC관광호텔]","[효명사, 고살리숲길, 망장포]","{'소개': '한라산과 바다를 전부 조망하는 단독형 펜션', '상세 정보': '체크..."
954,해뜨는집,,#휴식 #숙소 #펜션 #휴양펜션 #자연경관 #주차장 #힐링,제주특별자치도 제주시 서해안로 234,(+82) 064-742-7701,0,0,0,"1,373","해뜨는 집 전 객실에서 일출을 조망 할 수 있고,. 강중훈 시인이 직접 운영하며 노...","[서부두명품횟집거리, 산지항 옛터, 삼화포구]","[녹원전통찻집, 24시 뼈다귀탕 제주점, 국수와냉면]","[삼다펜션, 게스트하우스7, 호텔컬리넌제주]","[미래하우스, 레터스 투 시리얼, 빅토리아]","{'소개': '강중훈 시인이 운영하는 펜션', '이용 시간': 'Checkin : ..."


In [98]:
s = store_df.copy()

In [104]:
store_df.at[954, 'name' ] = '해뜨는집 (도두)'
store_df = store_df.drop(953, axis=0)
store_df = store_df.drop_duplicates('name').reset_index(drop=True)

In [105]:
store_df.to_csv('data/VISITJEJU_숙박크롤링_FINAL.csv', encoding='utf-8-sig', index=False)

## (2) 비짓제주 관광지 데이터 크롤링

In [60]:
url = 'https://www.visitjeju.net/kr/detail/list?menuId=DOM_000001707000000000&cate1cd=cate0000000004#p1&pageSize=12&sortListType=reviewcnt&viewType=thumb&isShowBtag&tag' #보여지는 형태를 꼭 네모 네개 짜리로 바꿀 것

In [8]:
driver = webdriver.Chrome(executable_path = 'D:\Python_417\chromedriver')
driver.get(url=url)

In [9]:
# implicit Waits
driver.implicitly_wait(time_to_wait = 5)

In [10]:
cleaning = re.compile('<p.*="">')

In [344]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
places = soup.select('dt>a>p.s_tit') # 제목
places = [cleaning.sub('', str(name))[:-4] for name in places]
# plc_tag = soup.find_all('p', class_= 'item_tag prev')
# plc_tag = [(cleaning.sub('', str(tags))[:-4]).strip() for tags in plc_tag]
# plc_tag는 안에 들어가면 더 있기 때문에 들어가서 구하는 걸로 하자

In [11]:
tmp = driver.find_element_by_css_selector('#content > div > div.cont_wrap > div.recommend_area > ul > li:nth-child(1) > dl > dt > a')
# 여기의 li:nth-child(1) 안의 숫자를 바꿔가면서 클릭하면 됨
tmp.click()

In [98]:
def click_get_info():
    twice = False
    try:
        name = driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.sub_info_title > h3').text
        stars = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.score_area_l > p').text[
              9:-1]
        tags = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.tag_area').text.replace(
            '\n', ' ')
        address = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(2) > p.info_sub_cont').text
        telephone = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(3) > p.info_sub_cont').text
        like = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(1) > button > p.appraisal_cnt').text
        heart = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(2) > button > p.appraisal_cnt').text
        numreview = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(3) > p.appraisal_cnt').text
        view = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(6) > p.appraisal_cnt').text
        ### 상세정보
        detail_text = driver.find_element_by_css_selector(
            '#tab0 > div').text.replace('\n', ' ')
        ### 추천콘텐츠
        #### 관광지
        rec_tour = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_contents').text.split('\n')[0:-1:2]
        #### 음식점
        try:
            driver.find_element_by_css_selector(
                '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_category > ul > li.add2020_detail_restaurant > a').click()
            rec_restaurant = driver.find_element_by_css_selector(
                '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_contents').text.split('\n')[0:-1:2]
        except:
            driver.find_element_by_css_selector('#tab0 > h2').click()
            driver.find_element_by_css_selector(
                '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_category > ul > li.add2020_detail_restaurant > a').click()
            rec_restaurant = driver.find_element_by_css_selector(
                '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_contents').text.split('\n')[0:-1:2]
            twice=True
        #### 숙박
        driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_category > ul > li.add2020_detail_lodgment > a').click()
        rec_lodgment = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_contents').text.split('\n')[0:-1:2]
        #### 빅데이터
        driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_category > ul > li.add2020_detail_bigdata > a').click()
        rec_bigdata = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_contents').text.split('\n')[0:-1:2]
        ### 이용안내
        detail_side_box = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(2) > div > dl').text.split('\n')
    except:
        name = driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.sub_info_title > h3').text
        stars = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style2 > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.score_area_l > p').text[9:-1]
        tags = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style2 > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.tag_area').text.replace('\n', ' ')
        tags+=' '+driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.tag_area > p:nth-child(2)').text.replace('\n','')
        address = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style2 > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(2) > p.info_sub_cont').text
        telephone = driver.find_element_by_css_selector(
             '#content > div.cont.detail_page.detail_style2 > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(3) > p.info_sub_cont').text
        ## 좋아요
        like = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > ul > li:nth-child(1) > button > p.appraisal_cnt').text
        ### 찜하기
        heart = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > ul > li:nth-child(2) > button > p.appraisal_cnt').text
        ### 리뷰
        numreview = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > ul > li:nth-child(3) > p.appraisal_cnt').text
        ### 조회
        view = driver.find_element_by_css_selector(
            '#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > ul > li:nth-child(6) > p.appraisal_cnt').text
        ### 상세정보
        detail_text = driver.find_element_by_css_selector('#tab0 > div').text.replace('\n', ' ')
        ### 추천콘텐츠
        #### 관광지
        rec_tour = driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_contents').text.split('\n')[0:-1:2]
        #### 음식점
        try:
            driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_category > ul > li.add2020_detail_restaurant > a').click()
            rec_restaurant = driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_contents').text.split('\n')[0:-1:2]
        except:
            driver.find_element_by_css_selector('#tab0 > h2').click()
            driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_category > ul > li.add2020_detail_restaurant > a').click()
            rec_restaurant = driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_contents').text.split('\n')[0:-1:2]
            twice=True
        #### 숙박
        driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_category > ul > li.add2020_detail_lodgment > a').click()
        rec_lodgment = driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_contents').text.split('\n')[0:-1:2]
        #### 빅데이터
        driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_category > ul > li.add2020_detail_bigdata > a').click()
        rec_bigdata = driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(1) > div.add2020_detail_side_contents').text.split('\n')[0:-1:2]
        ### 이용안내
        detail_side_box = driver.find_element_by_css_selector('#content > div.cont.detail_page.detail_style2 > div.cont_wrap.sub_visual > div.add2020_detail > div.add2020_detail_right > div:nth-child(2) > div > dl').text.split('\n')

    detail_side_box = { detail_side_box[i] : detail_side_box[i+1] for i in range(0,len(detail_side_box), 2) }

    return {'name' : name, 'stars' : stars, 'tags': tags, 'address' : address, 'telephone' : telephone, 'like' : like, 'heart' : heart, 'numreview' : numreview, 'view' : view, 'detail_text' : detail_text, 'rec_tour' : rec_tour, 'rec_restaurant' : rec_restaurant, 'rec_lodgment' :  rec_lodgment, 'rec_bigdata' : rec_bigdata, 'detail_side_box' : detail_side_box}, twice

In [99]:
driver.get(url=url)
driver.maximize_window()
driver.implicitly_wait(time_to_wait = 5)
cleaning = re.compile('<p.*="">')
final_df = pd.DataFrame(columns = ['name', 'stars', 'tags', 'address', 'telephone', 'like', 'heart', 'numreview', 'view', 'detail_text', 'rec_tour',
                                   'rec_restaurant', 'rec_lodgment', 'rec_bigdata', 'detail_side_box'])
page_number = int(driver.find_element_by_css_selector('#content > div > div.cont_wrap > div.recommend_area > div.theme_tit_area.clear > div > div > div.counter > p > strong').text)//12

In [100]:
def crawling_tour(driver, final_df):
    places = [element.text.split('\n')[1] for element in driver.find_elements_by_css_selector('#content > div > div.cont_wrap > div.recommend_area > ul > li')]
    for i in range(1,len(places)+1):
        place_click = driver.find_element_by_css_selector(f'#content > div > div.cont_wrap > div.recommend_area > ul > li:nth-child({i}) > dl > dt > a')
        place_click.click()
        driver.implicitly_wait(time_to_wait = 20)

        tmp_info_dict, twice = click_get_info()
        tmp_df = pd.DataFrame(columns = tmp_info_dict.keys(), data = [tmp_info_dict.values()])
        final_df = final_df.append(tmp_df)

        driver.execute_script("window.history.go(-1)")
        if twice==True:
            driver.execute_script("window.history.go(-1)")

        print(f'{i}번째 관광지')
    # driver.execute_script("window.history.go(-1)")
    return final_df

In [ ]:
for i in range(2,page_number):
    final_df = crawling_tour(driver, final_df)
    driver.find_element_by_css_selector('#paging > a.spr_com.page-next').click()
    driver.implicitly_wait(time_to_wait=10)
    final_df

1번째 관광지
2번째 관광지
3번째 관광지
4번째 관광지
5번째 관광지
6번째 관광지
7번째 관광지
8번째 관광지
9번째 관광지
10번째 관광지
11번째 관광지
12번째 관광지
1번째 관광지
2번째 관광지
3번째 관광지
4번째 관광지
5번째 관광지
6번째 관광지
7번째 관광지
8번째 관광지
9번째 관광지
10번째 관광지
11번째 관광지
12번째 관광지
1번째 관광지
2번째 관광지
3번째 관광지
4번째 관광지
5번째 관광지
6번째 관광지
7번째 관광지
8번째 관광지
9번째 관광지
10번째 관광지
11번째 관광지
12번째 관광지
1번째 관광지
2번째 관광지
3번째 관광지
4번째 관광지
5번째 관광지
6번째 관광지
7번째 관광지
8번째 관광지
9번째 관광지
10번째 관광지
11번째 관광지
12번째 관광지
1번째 관광지
2번째 관광지
3번째 관광지
4번째 관광지
5번째 관광지
6번째 관광지
7번째 관광지
8번째 관광지
9번째 관광지
10번째 관광지
11번째 관광지
12번째 관광지
1번째 관광지
2번째 관광지
3번째 관광지
4번째 관광지
5번째 관광지
6번째 관광지
7번째 관광지
8번째 관광지
9번째 관광지
10번째 관광지
11번째 관광지
12번째 관광지
1번째 관광지
2번째 관광지
3번째 관광지
4번째 관광지
5번째 관광지
6번째 관광지
7번째 관광지
8번째 관광지
9번째 관광지
10번째 관광지
11번째 관광지
12번째 관광지
1번째 관광지
2번째 관광지
3번째 관광지
4번째 관광지
5번째 관광지
6번째 관광지
7번째 관광지
8번째 관광지
9번째 관광지
10번째 관광지
11번째 관광지
12번째 관광지
1번째 관광지
2번째 관광지
3번째 관광지
4번째 관광지
5번째 관광지
6번째 관광지
7번째 관광지
8번째 관광지
9번째 관광지
10번째 관광지
11번째 관광지
12번째 관광지
13번째 관광지
14번째 관광지
15번째 관광지
16번째 관광지
17번째 관광지
18번째 관광지
19번째 관광지
20번째 관광지
21번째 관광지
22번째 관광지
23번째 관광지
24번째 관광지
2

In [50]:
final_df = final_df.drop_duplicates('name').reset_index(drop=True)

In [ ]:
final_df.to_csv('data/VISITJEJU_관광지정보크롤링_0914.csv', encoding='utf-8-sig', index=False)

## (3) 비짓제주 F&B 데이터 크롤링